# AF3-Vina-MMPBSA method for PFAS docking in proteins 

Xiping Gong (xipinggong@uga.edu) from the [Jack Huang's Lab](https://site.caes.uga.edu/huanglab/)

Date: 4/23/2025

# Introduction

Understanding how per- and polyfluoroalkyl substances (PFAS) interact with proteins at the molecular level is crucial for evaluating their biological impact. A reliable docking strategy is essential to accurately predict the protein-PFAS binding pose and to estimate the associated binding free energy. Such predictive models play an important role in identifying potential binding sites, understanding the binding mechanism, and guiding the design of safer chemical alternatives.

Recently, we proposed a hybrid docking strategy that combines AlphaFold 3 (AF3) with conventional AutoDock Vina to study PFAS-protein interactions. This AF3-Vina method leverages the strengths of both AI-based predictions and physics-based docking. By inputting only the protein sequence and ligand name, the method can predict likely binding pockets without requiring prior structural knowledge. This significantly reduces the search space for docking. For example, selecting the top five poses predicted by either AF3 or Vina results in a narrowed set of ten candidate poses, among which the correct binding pose can be likely identified with a success rate of over 80%.

To further refine the prediction and identify the most accurate binding pose, we apply the Molecular Mechanics/Poisson–Boltzmann Surface Area (MM/PBSA) method to calculate the binding free energies of the shortlisted poses. This step allows us to rank the predicted poses based on their binding free energies. In this tutorial, we present a step-by-step guide on how to apply the AF3-Vina-MM/PBSA method to dock PFAS into protein systems, enabling more accurate and efficient analysis of protein-PFAS interactions.


# Methodology

**Figure 1** illustrates how a combination of AF3 and AutoDock Vina predicts top binding poses of PFAS to proteins without prior knowledge of the binding pocket. The left panel shows five poses from AF3, and the right panel shows five from Vina. The top-ranked pose (red) is used to selected the candiate poses by calculating the RMSD values less than 0.2 nm. MM/PBSA is then applied to these top candidate poses to determine the most thermodynamically favorable binding conformation based on the calculated binding free energies.

<img src="images/af3_vina_mmpbsa.svg" alt="Alt Text" style="width:60%;">


**Figure 1**. Workflow of the AF3-Vina-MM/PBSA method for PFAS docking in proteins. This methods involves three steps: 1) using the AF3 method to obtain the candiate poses, 2) using the AF3Pocket-Vina method to obtain the candiate poses, and 3) using the MM/PBSA method to identify the best pose.


# Tutorial

In this section, I will show a step-by-step guide on how to use the AF3-Vina-MM/PBSA method. 
Here, we assume that we have installed all required packages, like Git, AF3, Vina, gmxMMPBSA, Python, and etc.


## Download the repository

I have created a GitHub repository, and we should have it downloaded first, which includes some required scripts. 

```bash
$ git clone https://github.com/XipingGong/pfas_docking.git
$ cd pfas_docking # go to this directory, and we will have a test later.
```


## Preparation of a protein-PFAS system

Here, we select a simple system (PDBID: 7FEU) as an example, which involves a simple protein system (a human heart fatty acid-binding protein), a perfluorononanoic acid (formula: C9HF17O2, short name: PFNA), and a hexaethylene glycol (formula: C12H26O7).

An illustrated description can be found in the following image (*Source: [RCSB PDB](https://www.rcsb.org/structure/7FEU)*):

<img src="https://cdn.rcsb.org/images/structures/7feu_assembly-1.jpeg" alt="Illustration of 7FEU" style="width:40%;">

To dock the PFNA into this human heart fatty acid-binding protein, we first need to download and clean up this PDBID, so that we have a system that only has one protein and one PFNA.

```bash
# Open a terminal and create a working directory and download the PDB file
$ mkdir -p test/dock_dir/7FEU_4I6 # create this folder: dock_dir/7FEU_4I6
$ cd test/dock_dir/7FEU_4I6 # go to the created folder
#
# Please modify this "../../../scripts/get_native_pdb.sh" file.
#
$ bash ../../../scripts/get_native_pdb.sh --pdbid 7FEU --ligandid 4I6 # It will create two pdb files
$ ls -ltr 
#>> 7FEU_4I6_ori.pdb
#>> 7FEU_4I6.pdb # this includes two chains: chain A for protein and chain B for ligand
$ mv 7FEU_4I6.pdb native_model.pdb # rename it as the native_model.pdb
```

After that, we will create the essential reference files for running the AF3-Vina-MMPBSA, including the native structure-derived pdb files, topology file, etc.

```bash
# Please modify this "../../../scripts/get_ref_for_af3vinammpbsa.sh" file.
#
$ mkdir -p native # create a native folder for saving the reference input
$ bash ../../../scripts/get_ref_for_af3vinammpbsa.sh --input_pdb native_model.pdb --work_dir native

```

## Running a job: AlphaFold 3

AF3 only requires a json file which includes the basic info of the protein-ligand complex, like protein sequence and ligand ID, so it can be straightforward to run the AlphaFold 3.

```bash
# Obtain the af3.sh script to run the AlphaFold 3
# Please modify this "../../../scripts/af3.sh" file.
# 
# - Create an "af3.json" file from a pdb file (it can be any pdb file has a protein-ligand system)
$ python ../../../scripts/get_json_for_af3.py native_model.pdb -o af3.json
#
# Submit an AF3 job
# - Case 1: we have the native one
$ sbatch ../../../scripts/af3.sh --input_json af3.json --native_dir native # submit a AF3 job. It needs the GPU installation
# - Case 2: we do not have the native one
$ sbatch ../../../scripts/af3.sh --input_json af3.json # we should provide a JSON file at least
#
# Check the result: we will see an "af3" folder, which includes all data generated
$ ls -ltr af3
# >> af3_data.json
# >> af3_model.cif
# >> af3_summary_confidences.json
# >> af3_confidences.json
# >> ranking_scores.csv
# >> TERMS_OF_USE.md
# >> best_pose
# >> seed-1_sample-0
# >> seed-1_sample-1
# >> seed-1_sample-2
# >> seed-1_sample-3
# >> seed-1_sample-4
# >> 
#
# Check out the predicted structrues
# It has five predicted structures. We can check their RMSD values, compared to the native one.
$ python ../../../scripts/check_rmsd.py --ref native/native_modelH.pdb "af3/[bs]*/aligned_model_convert.pdb"
# >. 📂 Loading reference PDB: native/native_modelH.pdb
# >.   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/native/native_modelH.pdb
# >. 📂 Loading target PDB: af3/[bs]*/aligned_model.pdb
# >.   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/af3/best_pose/aligned_model.pdb
# >.   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/af3/seed-1_sample-0/aligned_model.pdb
# >.   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/af3/seed-1_sample-1/aligned_model.pdb
# >.   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/af3/seed-1_sample-2/aligned_model.pdb
# >.   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/af3/seed-1_sample-3/aligned_model.pdb
# >.   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/af3/seed-1_sample-4/aligned_model.pdb
# >. 📊 Protein Backbone RMSD (Direct): Min = 0.023 nm ; [0.03027058 0.0234842  0.03139305 0.02682238 0.03312595 0.03027058] nm
# >. 📊 Protein Backbone RMSD (MDTraj): Min = 0.021 nm ; [0.02815207 0.0213471  0.0300446  0.02556344 0.03076157 0.02815207] nm
# >. 📊 Ligand RMSD (Direct): Min = 0.099 nm ; [0.12628981 0.12720062 0.1261062  0.12731826 0.09938771 0.12628981] nm
# >. 📊 Ligand RMSD (MDTraj): Min = 0.090 nm ; [0.12088684 0.12062304 0.12107167 0.12165862 0.09036002 0.12088684] nm
# >. 📊 Protein Backbone Pocket RMSD (Direct): Min = 0.017 nm ; [0.01763394 0.01680886 0.01733702 0.01748303 0.01763932 0.01763394] nm
# >. 📊 Protein Backbone Pocket RMSD (MDTraj): Min = 0.017 nm ; [0.01763385 0.01680982 0.0173394  0.01747647 0.01763385 0.01763385] nm
#
# Note: RMSD (Direct) is the RMSD value with a direct algorithm, and RMSD (MDTraj) is the RMSD value using the MDTraj that has a further structural alignment. RMSD < 0.2 nm is a good value to see how similar the predicted structures are.
```

## Running a job: AutoDock Vina

AutoDock Vina requires three input files: a receptor PDBQT file, a ligand PDBQT file, and binding pocket information (center coordinates and box size). The PDBQT format is an extension of the PDB file that includes torsion flexibility, partial charges, and AutoDock-specific atom types. To convert a standard receptor PDB file to PDBQT, the scripts (mk_prepare_receptor.py and mk_prepare_ligand.py) from [Meeko](https://meeko.readthedocs.io/en/release-doc/) can be used, which streamlines the process by adding hydrogens, assigning charges, and formatting the structure appropriately for docking.

```bash
# Obtain the vina.sh script to run the AutoDock Vina
# Please modify this "../../../scripts/vina.sh" file.
#
# Create a folder as a working directory to run a vina job 
$ mkdir -p vina
# Here, we use the AF3-predicted best one to submit a vina job
$ bash ../../../scripts/vina.sh --input_pdb af3/best_pose/aligned_model_convert.pdb --work_dir vina --native_dir native 
#
# Check the result: we will see an "vina" folder, which includes all data generated
$ ls -ltr vina
# >> vina_box_params.txt
# >> vina_receptor.pdb
# >> vina_ligand.pdb
# >> vina_receptor.pdbqt
# >> vina_ligand.sdf
# >> vina_ligand.pdbqt
# >> vina_ligand_docked.pdbqt
# >> vina_ligands.pdb
# >> vina_ligand_1.pdb
# >> vina_ligand_2.pdb
# >> vina_ligand_3.pdb
# >> vina_ligand_4.pdb
# >> vina_ligand_5.pdb
# >> vina_ligand_ref.pdb
# >> vina_ligand_1_convert.pdb
# >> vina_ligand_2_convert.pdb
# >> vina_ligand_3_convert.pdb
# >> vina_ligand_4_convert.pdb
# >> vina_ligand_5_convert.pdb
#
# Check out the predicted structrues
# It has five predicted structures. We can save them into the complex structures.
$ cat vina/vina_receptor.pdb vina/vina_ligand_1_convert.pdb | grep 'ATOM' > vina/vina_model_1_convert.pdb
$ cat vina/vina_receptor.pdb vina/vina_ligand_2_convert.pdb | grep 'ATOM' > vina/vina_model_2_convert.pdb
$ cat vina/vina_receptor.pdb vina/vina_ligand_3_convert.pdb | grep 'ATOM' > vina/vina_model_3_convert.pdb
$ cat vina/vina_receptor.pdb vina/vina_ligand_4_convert.pdb | grep 'ATOM' > vina/vina_model_4_convert.pdb
$ cat vina/vina_receptor.pdb vina/vina_ligand_5_convert.pdb | grep 'ATOM' > vina/vina_model_5_convert.pdb
# Then, we can check their RMSD values
$ python ../../../scripts/check_rmsd.py --ref native/native_modelH.pdb "vina/vina_model_*_convert.pdb"
# >> 📂 Loading reference PDB: native/native_modelH.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/native/native_modelH.pdb
# >> 📂 Loading target PDB: vina/vina_model_*.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/vina/vina_model_1.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/vina/vina_model_2.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/vina/vina_model_3.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/vina/vina_model_4.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/vina/vina_model_5.pdb
# >> 📊 Protein Backbone RMSD (Direct): Min = 0.030 nm ; [0.03027058 0.03027058 0.03027058 0.03027058 0.03027058] nm
# >> 📊 Protein Backbone RMSD (MDTraj): Min = 0.028 nm ; [0.02815207 0.02815207 0.02815207 0.02815207 0.02815207] nm
# >> 📊 Ligand RMSD (Direct): Min = 0.136 nm ; [0.48185012 0.4927522  0.13607053 0.5017956  0.4698871 ] nm
# >> 📊 Ligand RMSD (MDTraj): Min = 0.128 nm ; [0.15364015 0.17642318 0.1278825  0.12832251 0.15354529] nm
# >> 📊 Protein Backbone Pocket RMSD (Direct): Min = 0.018 nm ; [0.01763394 0.01763394 0.01763394 0.01763394 0.01763394] nm
# >> 📊 Protein Backbone Pocket RMSD (MDTraj): Min = 0.018 nm ; [0.01763385 0.01763385 0.01763385 0.01763385 0.01763385] nm
#
# Note: We see that the vina-predicted best pose is not the closest to the native one.
```



## Running a job: MM/PBSA

We will use the gmx_MMPBSA tool to calculate the binding free energy of a protein–ligand complex from a PDB structure. In our example, the PDB structure contains one protein and one ligand. To perform the gmx_MMPBSA calculations, specifically, we will apply the [single trajectory (ST) protocol](https://valdes-tresanco-ms.github.io/gmx_MMPBSA/dev/examples/Protein_ligand/ST).

It is necessary to have a look at [a ST example of gmx_MMPBSA](https://valdes-tresanco-ms.github.io/gmx_MMPBSA/dev/examples/Protein_ligand/ST). This example requires several inputs, including

+ **mmpbsa.in**: the input file of gmx_MMPBSA, which defines some parameters
+ **com.tpr**: this is tpr file from the GROMACS, which includes all possible info, including the force field, topology, etc.
+ **com_traj.xtc**: this is the trajectory file, which includes the coordinates of many trajectory frames. We assume that these trajectories are independently sampled and represent the major state of the protein-ligand complex.
+ **index.ndx**: this is the index file created from GROMACS, which includes the indices of protein and ligand "-cg 3 4" should refer to the protein and ligand in the index.ndx file.
+ **topol.top**: this is the topol.top file created from GROMACS, which includes all parameters, etc. 

To run a MM/PBSA job, we can start with a protein-ligand complex system predicted by either AF3 or Vina. 
This system could be missing hydrogen atoms, or not structurally optimized, so we will first add the hydrogen atoms and also do the geometry optimization, to obtain an optimized structure. After that, we will use the gmx_MMPBSA to calculate the binding free energy of this optmized strcuture.

```bash
# It is necessary to have a look at an example of gmx_MMPBSA.
# Please see here: https://valdes-tresanco-ms.github.io/gmx_MMPBSA/dev/examples/Protein_ligand/ST
#
# Collect all predicted protein-ligand model systems
$ ls -lrt af3/*/aligned_model_convert.pdb
# >> af3/best_pose/aligned_model_convert.pdb
# >> af3/seed-1_sample-0/aligned_model_convert.pdb
# >> af3/seed-1_sample-1/aligned_model_convert.pdb
# >> af3/seed-1_sample-2/aligned_model_convert.pdb
# >> af3/seed-1_sample-3/aligned_model_convert.pdb
# >> af3/seed-1_sample-4/aligned_model_convert.pdb
#
$ ls -lrt vina/vina_model_*_convert.pdb
# >> vina/vina_model_*_convert.pdb
# >> vina/vina_model_1_convert.pdb
# >> vina/vina_model_2_convert.pdb
# >> vina/vina_model_3_convert.pdb
# >> vina/vina_model_4_convert.pdb
# >> vina/vina_model_5_convert.pdb
#
# Here, we take the AF3-predicted best pose as an example: af3/best_pose/aligned_model_convert.pdb
# Add the hydrogen atoms
$ bash ../../../scripts/get_ref_for_af3vinammpbsa.sh --input_pdb af3/best_pose/aligned_model_convert.pdb --work_dir af3/best_pose --native_dir native # it will generate a pdb file: af3/best_pose/aligned_model_convertH.pdb
$ python ../../../scripts/check_2pdbs.py native/native_modelH.pdb af3/best_pose/aligned_model_convertH.pdb # Check this pdb file
$ python ../../../scripts/check_rmsd.py --ref native/native_modelH.pdb af3/best_pose/aligned_model_convertH.pdb # Check its RMSD values
#
# Optimize structure & run the MM/PBSA
$ mkdir -p mmpbsa
$ bash ../../../scripts/mmpbsa.sh --input_pdb af3/best_pose/aligned_model_convertH.pdb --work_dir mmpbsa --native_dir native
#
# Check the minimized structure
$ python ../../../scripts/check_2pdbs.py native/native_modelH.pdb mmpbsa/aligned_model_convertH_emin.pdb
$ python ../../../scripts/check_rmsd.py --ref native/native_modelH.pdb mmpbsa/aligned_model_convertH_emin.pdb
#
# Check the binding free energy
$ cat mmpbsa/aligned_model_convertH_emin_MMPBSA.dat # | grep 'ΔTOTAL'
# >> -------------------------------------------------------------------------------
# >> Delta (Complex - Receptor - Ligand):
# >> Energy Component       Average     SD(Prop.)         SD   SEM(Prop.)        SEM
# >> -------------------------------------------------------------------------------
# >> ΔBOND                    -0.00          0.00       0.00         0.00       0.00
# >> ΔANGLE                   -0.00          0.00       0.00         0.00       0.00
# >> ΔDIHED                   -0.00          0.00       0.00         0.00       0.00
# >> ΔVDWAALS                -35.83          0.00       0.00         0.00       0.00
# >> ΔEEL                    -65.18          0.00       0.00         0.00       0.00
# >> Δ1-4 VDW                 -0.00          0.00       0.00         0.00       0.00
# >> Δ1-4 EEL                 -0.00          0.00       0.00         0.00       0.00
# >> ΔEGB                     81.97          0.00       0.00         0.00       0.00
# >> ΔESURF                   -5.81          0.00       0.00         0.00       0.00
# >> 
# >> ΔGGAS                  -101.01          0.00       0.00         0.00       0.00
# >> ΔGSOLV                   76.16          0.00       0.00         0.00       0.00
# >> 
# >> ΔTOTAL                  -24.85          0.00       0.00         0.00       0.00
# >> -------------------------------------------------------------------------------
#
# Check the RMSD values
$ cat mmpbsa/aligned_model_convertH_emin_RMSD.dat 
# >> 📂 Loading reference PDB: /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/native/native_modelH.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/native/native_modelH.pdb
# >> 📂 Loading target PDB: aligned_model_convertH_emin.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7FEU_4I6/mmpbsa/aligned_model_convertH_emin.pdb
# >> 📊 Protein Backbone RMSD (Direct): Min = 0.030 nm ; [0.03020584] nm
# >> 📊 Protein Backbone RMSD (MDTraj): Min = 0.028 nm ; [0.02817651] nm
# >> 📊 Ligand RMSD (Direct): Min = 0.125 nm ; [0.1249753] nm
# >> 📊 Ligand RMSD (MDTraj): Min = 0.120 nm ; [0.11951476] nm
# >> 📊 Protein Backbone Pocket RMSD (Direct): Min = 0.021 nm ; [0.02059139] nm
# >> 📊 Protein Backbone Pocket RMSD (MDTraj): Min = 0.021 nm ; [0.0205908] nm
#
# To have a batch processing, we can run the following script:
$ bash run_all_mmpbsa.sh | tee run_all_mmpbsa.out
# Then, we can check their MMPBSA.dat and RMSD.dat
$ grep 'ΔTOTAL' mmpbsa/*_MMPBSA.dat
# >> mmpbsa/af3-best_pose-aligned_model_convertH_emin_MMPBSA.dat:ΔTOTAL                  -24.85          0.00       0.00         0.00       0.00
# >> mmpbsa/af3-seed-1_sample-0-aligned_model_convertH_emin_MMPBSA.dat:ΔTOTAL                  -23.75          0.00       0.00         0.00       0.00
# >> mmpbsa/af3-seed-1_sample-1-aligned_model_convertH_emin_MMPBSA.dat:ΔTOTAL                  -24.65          0.00       0.00         0.00       0.00
# >> mmpbsa/af3-seed-1_sample-2-aligned_model_convertH_emin_MMPBSA.dat:ΔTOTAL                  -25.04          0.00       0.00         0.00       0.00
# >> mmpbsa/af3-seed-1_sample-3-aligned_model_convertH_emin_MMPBSA.dat:ΔTOTAL                  -24.96          0.00       0.00         0.00       0.00
# >> mmpbsa/af3-seed-1_sample-4-aligned_model_convertH_emin_MMPBSA.dat:ΔTOTAL                  -24.85          0.00       0.00         0.00       0.00
# >> mmpbsa/vina-vina_model_1_convertH_emin_MMPBSA.dat:ΔTOTAL                  -23.39          0.00       0.00         0.00       0.00
# >> mmpbsa/vina-vina_model_2_convertH_emin_MMPBSA.dat:ΔTOTAL                  -18.17          0.00       0.00         0.00       0.00
# >> mmpbsa/vina-vina_model_3_convertH_emin_MMPBSA.dat:ΔTOTAL                  -21.32          0.00       0.00         0.00       0.00
# >> mmpbsa/vina-vina_model_4_convertH_emin_MMPBSA.dat:ΔTOTAL                  -23.26          0.00       0.00         0.00       0.00
# >> mmpbsa/vina-vina_model_5_convertH_emin_MMPBSA.dat:ΔTOTAL                  -24.16          0.00       0.00         0.00       0.00
#
$ grep 'Ligand RMSD (Direct)' mmpbsa/*_Top1RMSD.dat
# >> mmpbsa/af3-best_pose-aligned_model_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.000 nm ; [0.] nm
# >> mmpbsa/af3-seed-1_sample-0-aligned_model_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.008 nm ; [0.00846293] nm
# >> mmpbsa/af3-seed-1_sample-1-aligned_model_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.116 nm ; [0.11583543] nm
# >> mmpbsa/af3-seed-1_sample-2-aligned_model_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.082 nm ; [0.08212418] nm
# >> mmpbsa/af3-seed-1_sample-3-aligned_model_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.116 nm ; [0.11610258] nm
# >> mmpbsa/af3-seed-1_sample-4-aligned_model_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.000 nm ; [0.] nm
# >> mmpbsa/vina-vina_model_1_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.000 nm ; [0.] nm
# >> mmpbsa/vina-vina_model_2_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.293 nm ; [0.29329646] nm
# >> mmpbsa/vina-vina_model_3_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.487 nm ; [0.48655078] nm
# >> mmpbsa/vina-vina_model_4_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.200 nm ; [0.2003686] nm
# >> mmpbsa/vina-vina_model_5_convertH_emin_Top1RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.170 nm ; [0.17047724] nm
#
$ grep 'Ligand RMSD (Direct)' mmpbsa/*_RMSD.dat
# >> mmpbsa/af3-best_pose-aligned_model_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.125 nm ; [0.1249753] nm
# >> mmpbsa/af3-seed-1_sample-0-aligned_model_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.126 nm ; [0.12637609] nm
# >> mmpbsa/af3-seed-1_sample-1-aligned_model_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.124 nm ; [0.12439268] nm
# >> mmpbsa/af3-seed-1_sample-2-aligned_model_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.126 nm ; [0.12602165] nm
# >> mmpbsa/af3-seed-1_sample-3-aligned_model_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.097 nm ; [0.09743661] nm
# >> mmpbsa/af3-seed-1_sample-4-aligned_model_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.125 nm ; [0.1249753] nm
# >> mmpbsa/aligned_model_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.125 nm ; [0.1249753] nm
# >> mmpbsa/vina-vina_model_1_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.487 nm ; [0.48672646] nm
# >> mmpbsa/vina-vina_model_2_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.498 nm ; [0.4978356] nm
# >> mmpbsa/vina-vina_model_3_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.137 nm ; [0.13723528] nm
# >> mmpbsa/vina-vina_model_4_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.502 nm ; [0.50161225] nm
# >> mmpbsa/vina-vina_model_5_convertH_emin_RMSD.dat:📊 Ligand RMSD (Direct): Min = 0.473 nm ; [0.47266927] nm
#
```



In [ ]:
from IPython.display import Markdown, display
import requests

# Use the raw GitHub URL
url = "https://raw.githubusercontent.com/XipingGong/pfas_docking/main/scripts/run_all_mmpbsa.sh"
code = requests.get(url).text

# Display with syntax highlighting
display(Markdown(f"```bash\n{code}\n```"))

# Appendix


## Scripts

### get_native_pdb.sh


In [4]:
from IPython.display import Markdown, display
import requests

# Use the raw GitHub URL
url = "https://raw.githubusercontent.com/XipingGong/pfas_docking/main/scripts/get_native_pdb.sh"
code = requests.get(url).text

# Display with syntax highlighting
display(Markdown(f"```bash\n{code}\n```"))


```bash
404: Not Found
```

### get_ref_for_af3vinammpbsa.sh


In [5]:
from IPython.display import Markdown, display
import requests

# Use the raw GitHub URL
url = "https://raw.githubusercontent.com/XipingGong/pfas_docking/main/scripts/get_ref_for_af3vinammpbsa.sh"
code = requests.get(url).text

# Display with syntax highlighting
display(Markdown(f"```bash\n{code}\n```"))


```bash
404: Not Found
```

### af3.sh


In [5]:
from IPython.display import Markdown, display
import requests

# Use the raw GitHub URL
url = "https://raw.githubusercontent.com/XipingGong/pfas_docking/main/scripts/af3.sh"
code = requests.get(url).text

# Display with syntax highlighting
display(Markdown(f"```bash\n{code}\n```"))


```bash
404: Not Found
```

### vina.sh

In [5]:
from IPython.display import Markdown, display
import requests

# Use the raw GitHub URL
url = "https://raw.githubusercontent.com/XipingGong/pfas_docking/main/scripts/vina.sh"
code = requests.get(url).text

# Display with syntax highlighting
display(Markdown(f"```bash\n{code}\n```"))


```bash
404: Not Found
```

### mmpbsa.sh


In [5]:
from IPython.display import Markdown, display
import requests

# Use the raw GitHub URL
url = "https://raw.githubusercontent.com/XipingGong/pfas_docking/main/scripts/mmpbsa.sh"
code = requests.get(url).text

# Display with syntax highlighting
display(Markdown(f"```bash\n{code}\n```"))


```bash
404: Not Found
```